In [141]:
import numpy as np
import math

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from pint import UnitRegistry

## constants

- source: fluid properties https://www.climate-policy-watcher.org/wastewater-sludge/physical-and-biological-properties.html

In [176]:
u = UnitRegistry()
kg = u.kilogram
m = u.meter 
s = u.second


In [267]:
def constants(rho, mu,  D, h, beta, override=False, perc_grade=0.015, alpha=2, g=9.8):

    # -----> to calculate pipe length, L
    
    h =  h * m
    beta = beta * m 
    H = h + beta 

    # max grade, slope s
    perc_grade = 0.015 

    # pipe horz distance travelled, x 
    x = h / perc_grade

    # angle required, theta 
    # TODO check if this is h/x or x/h 
    theta = np.rad2deg(np.arctan(x/h))

    # pipe length, L
    L = x / np.sin(theta) # m
    
    c = {
        # --- fluid ---------------> 
        # density of primary sludge
        "rho": rho * kg / (m**3),
        # dynamic viscosity
        "mu": mu * kg / (m*s),
        # ---- infra ------------->
        # pipe diameter 
        "D": D * m,  
        # lift height, h 
        "h": h,
        # tank height, beta 
        "beta": beta,
        # total height dif
        "H": H,
        # total horz distance 
        "x ": x,
        # angle 
        "theta": theta,
        # pipe length 
        "L": L,
        # ---- physics ----------->
        "g": g * m / (s**2),
        "alpha": alpha
        
    }

    if override:
        # use values from the Cengel example 8.1
        c["rho"] = 998 * kg / (m**3)
        c["mu"] = 0.00102 * kg / (m*s)
        c["D"] = 0.06 * m  
        c["L"] = 65 * m 
        c["H"] = 2.20 * m 

    return c



In [245]:
# example constants 
ex_c = constants(1, 1,  1, 1, 1, override=True)
ex_c

{'rho': 998.0 <Unit('kilogram / meter ** 3')>,
 'mu': 0.00102 <Unit('kilogram / meter / second')>,
 'D': 0.06 <Unit('meter')>,
 'h': 1 <Unit('meter')>,
 'beta': 1 <Unit('meter')>,
 'H': 2.2 <Unit('meter')>,
 'x ': 66.66666666666667,
 'theta': 89.14062775635531,
 'L': 65 <Unit('meter')>,
 'g': 9.8 <Unit('meter / second ** 2')>,
 'alpha': 2}

In [268]:
# real constants 
# density of primary sludge => 1.0 -> 1.03 g/cm3 =>=> 1000 kg/m3 -> 1030 kg/m3
# dynamic viscosity => 0.5 and 2 Pa*s  ~ kg/(m*s)
rc = constants(1000, 0.5, 0.1, 1.1, 1.1 )
rc

{'rho': 1000.0 <Unit('kilogram / meter ** 3')>,
 'mu': 0.5 <Unit('kilogram / meter / second')>,
 'D': 0.1 <Unit('meter')>,
 'h': 1.1 <Unit('meter')>,
 'beta': 1.1 <Unit('meter')>,
 'H': 2.2 <Unit('meter')>,
 'x ': 73.33333333333334 <Unit('meter')>,
 'theta': 89.14062775635531 <Unit('degree')>,
 'L': 73.34158286932303 <Unit('meter')>,
 'g': 9.8 <Unit('meter / second ** 2')>,
 'alpha': 2}

# calculations

In [270]:
# set constants 
d = rc

In [271]:
a = 1
b = (64 * d["mu"] * d["L"] ) / (d["rho"] * d["D"]**2 * d["alpha"] )
c =  -(2 * d["g"] * d["H"]) / (d["alpha"])

coeff = {
    "a": a,
    "b": b,
    "c": c
}

print(coeff)

{'a': 1, 'b': <Quantity(117.346533, 'meter / second')>, 'c': <Quantity(-21.56, 'meter ** 2 / second ** 2')>}


## plot

In [276]:
x =  np.linspace(-100, 100, 100)  * m / s

y = (a * x**2) + (b * x) + c

y[0]


-1756.213259091683 <Unit('meter ** 2 / second ** 2')>

In [277]:

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=x.magnitude,
    y=y.magnitude, 
    mode='lines+markers',
))

fig.update_layout(title='Examine Quad - Solving for V',
                   xaxis_title='x ',
                   yaxis_title='y')

In [278]:
def solve_quad(a, b, c):
    # a = a
    # b = b.magnitude
    # c = c.magnitude
    print(a,b,c)
    d = b ** 2 - 4 * a * c  # this part is called the discriminant

    if d < 0:
        print("The equation has no real solutions")
    elif d == 0:
        x = (-b + np.sqrt(b ** 2 - 4 * a * c)) / (2 * a)
        print(f"The equation has one solution: {x} ")
        return x
    else:
        x1 = (-b + np.sqrt(b ** 2 - 4 * a * c)) / (2 * a)
        x2 = (-b - np.sqrt(b ** 2 - 4 * a * c)) / (2 * a)
        print(f"The equation has two solutions: {x1} or {x2}")
        return (x1,x2)
    

In [279]:
vel_result = solve_quad(a, b, c)

vel_result 

1 117.34653259091684 meter / second -21.560000000000002 meter ** 2 / second ** 2
The equation has two solutions: 0.18344256409111637 meter / second or -117.52997515500795 meter / second


(0.18344256409111637 <Unit('meter / second')>,
 -117.52997515500795 <Unit('meter / second')>)

In [295]:
vel = vel_result[0]
vel

0.18344256409111637 <Unit('meter / second')>

## volume flow rate

In [283]:
# # vfr , volume flow rate, m3/s
vfr = (vel* math.pi * d["D"]**2) / 4
vfr

0.0014407545292608151 <Unit('meter ** 3 / second')>

## work from the pump 

In [284]:
Re = (d["rho"]*vel*d["D"]) / d["mu"]
Re

36.68851281822327 <Unit('dimensionless')>

In [286]:
f = 64/ Re
f

1.7444152156587565 <Unit('dimensionless')>

In [296]:
vel**2

0.033651174320323336 <Unit('meter ** 2 / second ** 2')>

In [304]:
h_l = f * (d["L"] / d["D"]) *  0.5*((vel**2) / d["g"])
h_l

2.196566206701994 <Unit('meter')>

In [305]:
# rate of pump work -> kg m2/s^3 (Watts)
w_pump = vfr * d["rho"] * d["g"] * h_l
w_pump

31.01418456904603 <Unit('kilogram * meter ** 2 / second ** 3')>